In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from dateutil.parser import parse
import glob
import sys
import openpyxl as opx
import xlrd

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
%matplotlib inline

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed,GRU, Conv1D

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 데이터 추출

In [2]:
# kepsh1 엑셀 데이터 추출

files = glob.glob('./kepsh1/*.xls')

all_data = pd.DataFrame()
for f in files:
    data = pd.read_csv(f, encoding='euc-kr', engine= 'python',  header=0, sep='\t+')
    
    df = pd.DataFrame(data)
    
    all_data = all_data.append(df, ignore_index=True)
    print(f, all_data.shape)
all_data


./kepsh1\2020-06-11 12시 08분 35초.xls (94, 35)
./kepsh1\2020-06-11 13시 44분 04초.xls (97, 35)
./kepsh1\2020-06-11 13시 45분 57초.xls (101, 35)
./kepsh1\2020-06-11 13시 56분 03초.xls (103, 35)
./kepsh1\2020-06-11 13시 57분 20초.xls (103, 35)
./kepsh1\2020-06-11 13시 58분 23초.xls (705, 35)
./kepsh1\2020-06-12 00시 00분 07초.xls (2145, 35)
./kepsh1\2020-06-13 00시 00분 07초.xls (3585, 35)
./kepsh1\2020-06-14 00시 00분 06초.xls (5025, 35)
./kepsh1\2020-06-15 00시 00분 06초.xls (6465, 35)
./kepsh1\2020-06-16 00시 00분 05초.xls (7905, 35)
./kepsh1\2020-06-17 00시 00분 05초.xls (8566, 35)
./kepsh1\Z1_2020-06-17 11시 03분 47초.xls (8567, 35)
./kepsh1\Z1_2020-06-17 11시 04분 04초.xls (8620, 35)
./kepsh1\Z1_2020-06-17 11시 56분 28초.xls (9344, 35)
./kepsh1\Z1_2020-06-18 00시 00분 03초.xls (10784, 35)
./kepsh1\Z1_2020-06-19 00시 00분 03초.xls (12224, 35)
./kepsh1\Z1_2020-06-20 00시 00분 03초.xls (13664, 35)
./kepsh1\Z1_2020-06-21 00시 00분 02초.xls (15104, 35)
./kepsh1\Z1_2020-06-22 00시 00분 02초.xls (16544, 35)
./kepsh1\Z1_2020-06-23 00시 00분 01초.xls 

./kepsh1\Z1_2020-12-03 16시 51분 16초.xls (179868, 35)
./kepsh1\Z1_2020-12-04 00시 00분 34초.xls (181308, 35)
./kepsh1\Z1_2020-12-05 00시 00분 33초.xls (182748, 35)
./kepsh1\Z1_2020-12-06 00시 00분 32초.xls (184188, 35)
./kepsh1\Z1_2020-12-07 00시 00분 30초.xls (185628, 35)
./kepsh1\Z1_2020-12-08 00시 00분 29초.xls (187068, 35)
./kepsh1\Z1_2020-12-09 00시 00분 28초.xls (187892, 35)
./kepsh1\Z1_2020-12-09 13시 44분 42초.xls (188508, 35)
./kepsh1\Z1_2020-12-10 00시 00분 27초.xls (189948, 35)
./kepsh1\Z1_2020-12-11 00시 00분 25초.xls (191388, 35)
./kepsh1\Z1_2020-12-12 00시 00분 24초.xls (192828, 35)
./kepsh1\Z1_2020-12-13 00시 00분 23초.xls (194268, 35)
./kepsh1\Z1_2020-12-14 00시 00분 22초.xls (195708, 35)
./kepsh1\Z1_2020-12-15 00시 00분 20초.xls (197148, 35)
./kepsh1\Z1_2020-12-16 00시 00분 19초.xls (198588, 35)
./kepsh1\Z1_2020-12-17 00시 00분 19초.xls (200028, 35)
./kepsh1\Z1_2020-12-18 00시 00분 18초.xls (201468, 35)
./kepsh1\Z1_2020-12-19 00시 00분 16초.xls (202908, 35)
./kepsh1\Z1_2020-12-20 00시 00분 15초.xls (204348, 35)
./kepsh1\Z1_

./kepsh1\Z1_2021-03-03 00시 00분 02초.xls (308029, 91)
./kepsh1\Z1_2021-03-03 14시 52분 57초.xls (308046, 91)
./kepsh1\Z1_2021-03-03 15시 37분 41초.xls (308047, 91)
./kepsh1\Z1_2021-03-03 15시 38분 48초.xls (308549, 91)
./kepsh1\Z1_2021-03-04 00시 00분 06초.xls (309989, 91)
./kepsh1\Z1_2021-03-05 00시 00분 06초.xls (311429, 91)
./kepsh1\Z1_2021-03-06 00시 00분 06초.xls (312869, 91)
./kepsh1\Z1_2021-03-07 00시 00분 04초.xls (314309, 91)
./kepsh1\Z1_2021-03-08 00시 00분 02초.xls (314983, 91)
./kepsh1\Z1_2021-03-08 11시 15분 18초.xls (315748, 91)
./kepsh1\Z1_2021-03-09 00시 00분 01초.xls (316417, 91)
./kepsh1\Z1_2021-03-09 11시 09분 23초.xls (316533, 91)
./kepsh1\Z1_2021-03-09 13시 07분 36초.xls (317186, 91)
./kepsh1\Z1_2021-03-10 00시 00분 00초.xls (318627, 91)
./kepsh1\Z1_2021-03-11 00시 00분 59초.xls (320067, 91)
./kepsh1\Z1_2021-03-12 00시 00분 57초.xls (321507, 91)
./kepsh1\Z1_2021-03-13 00시 00분 56초.xls (322947, 91)
./kepsh1\Z1_2021-03-14 00시 00분 55초.xls (324387, 91)
./kepsh1\Z1_2021-03-15 00시 00분 54초.xls (325827, 91)
./kepsh1\Z1_

./kepsh1\Z1_2021-06-27 00시 00분 51초.xls (470330, 195)
./kepsh1\Z1_2021-06-28 00시 00분 50초.xls (471770, 195)
./kepsh1\Z1_2021-06-29 00시 00분 51초.xls (473210, 195)
./kepsh1\Z1_2021-06-30 00시 00분 49초.xls (473584, 195)
./kepsh1\Z1_2021-06-30 06시 14분 05초.xls (473585, 196)
./kepsh1\Z1_2021-06-30 06시 16분 58초.xls (474216, 196)
./kepsh1\Z1_2021-06-30 17시 02분 19초.xls (474217, 196)
./kepsh1\Z1_2021-06-30 17시 05분 32초.xls (474445, 196)
./kepsh1\Z1_2021-07-01 10시 50분 39초.xls (474463, 196)
./kepsh1\Z1_2021-07-01 11시 08분 47초.xls (475235, 196)
./kepsh1\Z1_2021-07-02 00시 00분 47초.xls (476675, 196)
./kepsh1\Z1_2021-07-03 00시 00분 46초.xls (478115, 196)
./kepsh1\Z1_2021-07-04 00시 00분 44초.xls (479555, 196)
./kepsh1\Z1_2021-07-05 00시 00분 43초.xls (480551, 196)
./kepsh1\Z1_2021-07-05 16시 38분 11초.xls (480552, 196)
./kepsh1\Z1_2021-07-05 16시 40분 50초.xls (480658, 196)


,Time,1.Z1_메인(W),1.Z1_메인(kWh),2.Z1_B1구히트(W),2.Z1_B1구히트(kWh),3.Z1_B1조명(W),3.Z1_B1조명(kWh),4.Z1_B1전열(W),4.Z1_B1전열(kWh),5.Z1_펌프(W),...,KP1_8DC(kW),KP1_8AC(V),KP1_8AC(A),KP1_8AC(kW),KP1_8(kWh),연료전지소비(W),연료전지소비(kWh),연료전지생산(W),연료전지생산(kWh),경사면일사량(W/㎡)
0,2020-06-11 오후 12:08:35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-11 오후 12:09:06,-530.0,227.6,0.0,16.0,0.0,13.7,4.0,115.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-11 오후 12:10:06,-653.0,227.6,0.0,16.0,0.0,13.7,2.0,115.0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-06-11 오후 12:11:06,-369.0,227.6,0.0,16.0,0.0,13.7,2.0,115.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-06-11 오후 12:12:06,-132.0,227.6,0.0,16.0,0.0,13.7,2.0,115.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480653,2021-07-05 18:21:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,93.0,102.1,0.0,304.9,10.5
480654,2021-07-05 18:22:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,93.0,102.1,0.0,304.9,11.4
480655,2021-07-05 18:23:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,112.0,102.1,0.0,304.9,12.4
480656,2021-07-05 18:24:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,91.0,102.1,0.0,304.9,13.2


In [ ]:
all_features = all_data.columns.tolist()
print(all_features)

In [ ]:
# Time 컬럼에서 오전, 오후 한글은 pd.to_date 적용 불가 AM, PM으로 각각 변경

all_data['Time'] = all_data['Time'].replace({'오전': 'AM', '오후':'PM'}, regex=True)

In [ ]:
# 시간만 따로 떼서 시간 포맷 바뀌는 지점 확인
all_data['Time'].to_csv('kepsh1_time.csv', encoding='euc-kr')

In [ ]:
# Time 컬럼 내 시각형식 다른 두 컬럼 분리 

all_data_1 = all_data.iloc[0:372597,:]
all_data_2 = all_data.iloc[372597:, :]

In [ ]:
all_data_1.head()

In [ ]:
# all_data_1는 포맷변경 적용 필요

all_data_1['Time']= pd.to_datetime(all_data_1['Time'], errors = 'coerce', format = "%Y-%m-%d %p %I:%M:%S")
all_data_1.head()

In [ ]:
# all_data_2는 포맷변경 없이 바로 적용 가능

all_data_2['Time']= pd.to_datetime(all_data_2['Time'])
all_data_2.info()

In [ ]:
all_data_2.head()

In [ ]:
# Time 컬럼 포맷 변경 후 다시 concat
all_data_3 = pd.concat([all_data_1, all_data_2])
all_data_3.info()

In [ ]:
all_data_3

In [ ]:
# 데이터프레임 저장
all_data_3.to_csv('./kepsh1_concat.csv', encoding='euc=kr')

# 합쳐진 데이터 로드

In [ ]:
df = pd.read_csv('./kepsh1_concat.csv', encoding='euc-kr', parse_dates = True, index_col = 'Time')

In [ ]:
df.drop(['Unnamed: 0'], inplace = True, axis =1 )
df.head()

# PV 데이터 concat

In [ ]:
df['KP1_PV(W)'] = (df['Z1_1AC(V)']*df['Z1_1AC(A)'] + df['Z1_2AC(V)']*df['Z1_2AC(A)'] + df['Z1_3AC(V)']*df['Z1_3AC(A)'] +
                 df['Z1_4AC(V)']*df['Z1_4AC(A)'] + df['Z1_5AC(V)']*df['Z1_5AC(A)'] + df['Z1_6AC(V)']*df['Z1_6AC(A)'] + 
                 df['KP1_1AC(V)']*df['KP1_1AC(A)'] + df['KP1_2AC(V)']*df['KP1_2AC(A)'] + df['KP1_3AC(V)']*df['KP1_3AC(A)'] +
                 df['KP1_4AC(V)']*df['KP1_4AC(A)'] + df['KP1_5AC(V)']*df['KP1_5AC(A)'] + df['KP1_6AC(V)']*df['KP1_6AC(A)'])

In [ ]:
df['Z1_PV(W)'] = (df['Z1_1AC(V)']*df['Z1_1AC(A)'] + df['Z1_2AC(V)']*df['Z1_2AC(A)'] + df['Z1_3AC(V)']*df['Z1_3AC(A)'] +
                 df['Z1_4AC(V)']*df['Z1_4AC(A)'] + df['Z1_5AC(V)']*df['Z1_5AC(A)'] + df['Z1_6AC(V)']*df['Z1_6AC(A)'])
                

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(df['Z1_PV(W)'])
plt.show()


In [ ]:
df['KP1_PV(W)'] = (df['KP1_1AC(V)']*df['KP1_1AC(A)'] + df['KP1_2AC(V)']*df['KP1_2AC(A)'] + df['KP1_3AC(V)']*df['KP1_3AC(A)'] +
                 df['KP1_4AC(V)']*df['KP1_4AC(A)'] + df['KP1_5AC(V)']*df['KP1_5AC(A)'] + df['KP1_6AC(V)']*df['KP1_6AC(A)'])

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(df['KP1_PV(W)'])
plt.show()


In [ ]:
df['Z1_PV(W)'].dropna()

In [ ]:
df['KP1_PV(W)'].dropna()

In [ ]:
df['PV2(W)'] = df['Z1_PV(W)'] + df['KP1_PV(W)']

In [ ]:
df['PV3(W)'] = pd.merge([df['Z1_PV(W)'],df['KP1_PV(W)']], )

In [ ]:
df['PV2(W)']

In [ ]:
plt.figure(figsize=(18,6))
plt.plot(df['Z1_PV(W)'])
plt.show()



# 데이터 분석

In [ ]:
# 시간별 데이터 분석
fig, ax = plt.subplots(figsize=(18,700))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    name = df.columns[i]
    plt.plot(df[name]) 
    plt.title(name, loc = 'left', pad=5)   
    plt.yticks()
    plt.grid(True)
plt.show()
fig.tight_layout()

In [ ]:
df.describe()